# `spacy`: el *Ruby on Rails* del PLN

[spacy](http://www.spacy.io/) es una librería de procesamiento del lenguaje natural, robusta, rápida, fácil de instalar y utilizar e integrable con [otras librerías de *NLP* y de *deep learning*](https://spacy.io/usage/facts-figures#section-other-libraries). 

Tiene modelos entrenados en varios idiomas y permite realizar las [típicas tareas](https://spacy.io/usage/facts-figures) de segmentación por oraciones, tokenizanción, análisis morfológico, extracción de entidades y análisis de opinión.

Veamos cómo funciona. Lo primero es instalar algunos modelos, en inglés y español. Desde la línea de comandos, ejecuta:

In [1]:
#!python3 -m spacy download en
#!python3 -m spacy download es

In [2]:
import spacy

nlp = spacy.load("es")

Una vez instalados los modelos, podemos importarlos fácilmente:

In [3]:
import spacy

# cargamos el modelo entrenado en español
nlp = spacy.load("es")

texto = """España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 
España no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que 
suspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario 
del Tratado de Maastricht y obtiene así más margen de gasto en 2018."""

# y procesamos el texto
doc = nlp(texto)

## Procesando oraciones, palabras y entidades

Podemos iterar fácilmente sobre la lista de oraciones y recorrer los tokens para acceder a su información morfo-sintáctica:

In [4]:
for sentence in doc.sents:
    print("Oración: {}".format(sentence))
    for token in sentence:
        print("{}/{} => etiqueta {}/{} y dependencia {}".format(token, token.lemma_, token.pos_, token.tag_, token.dep_))

Oración: España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. 

España/España => etiqueta PROPN/PROPN___ y dependencia nsubj
incumple/incumplir => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia ROOT
la/lo => etiqueta DET/DET__Definite=Def|Gender=Fem|Number=Sing|PronType=Art y dependencia det
regla/reglar => etiqueta NOUN/NOUN__Gender=Fem|Number=Sing y dependencia obj
del/del => etiqueta ADP/ADP__AdpType=Preppron|Gender=Masc|Number=Sing y dependencia case
déficit/déficit => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
del/del => etiqueta ADP/ADP__AdpType=Preppron|Gender=Masc|Number=Sing y dependencia case
euro/euro => etiqueta NOUN/NOUN__Gender=Masc|Number=Sing y dependencia nmod
y/y => etiqueta CONJ/CCONJ___ y dependencia cc
queda/quedo => etiqueta VERB/VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin y dependencia conj
como/comer => etiqueta SCONJ/SCONJ___ y dependencia 

El elemeno `doc` tiene una propiedad `.ents` que permite acceder a las entidades nombradas que hayan sido localizadas:

In [5]:
print(doc.ents)

for entity in doc.ents:
    print("{} es de tipo {}".format(entity, entity.label_))

(Bruselas, Eurozona, El Gobierno, Tratado de Maastricht)
Bruselas es de tipo LOC
Eurozona es de tipo LOC
El Gobierno es de tipo LOC
Tratado de Maastricht es de tipo MISC


## Visualizando árboles de dependencias y entidades

Podemos acceder al árbol completo a través del método `.to_json()`:

In [6]:
doc.to_json()

{'text': 'España incumple la regla del déficit del euro y queda como único país bajo el control de Bruselas. \nEspaña no aprobará finalmente la regla europa del déficit y se quedará como único país de la Eurozona que \nsuspende y sigue bajo vigilancia. El Gobierno se salta finalmente el requisito de saneamiento presupuestario \ndel Tratado de Maastricht y obtiene así más margen de gasto en 2018.',
 'ents': [{'start': 89, 'end': 97, 'label': 'LOC'},
  {'start': 193, 'end': 201, 'label': 'LOC'},
  {'start': 241, 'end': 252, 'label': 'LOC'},
  {'start': 321, 'end': 342, 'label': 'MISC'}],
 'sents': [{'start': 0, 'end': 100},
  {'start': 100, 'end': 240},
  {'start': 241, 'end': 385}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 6,
   'pos': 'PROPN',
   'tag': 'PROPN___',
   'dep': 'nsubj',
   'head': 1},
  {'id': 1,
   'start': 7,
   'end': 15,
   'pos': 'VERB',
   'tag': 'VERB__Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin',
   'dep': 'ROOT',
   'head': 1},
  {'id': 2,
   'sta

Pero también se puede dibujar el grafo con las dependencias:

In [ ]:
from spacy import displacy
displacy.serve(doc, style="dep")

O el de las entidades:

In [ ]:
from spacy import displacy
displacy.serve(doc, style="ent")

In [ ]:
texto = """Los ciudadanos españoles son conscientes de que esa salida de Reino Unido no será gratuita. Literalmente. Uno de cada tres teme que el Brexit le cueste caro a la economía española: el 36,9% prevé consecuencias «muy negativas» para sus bolsillos."""
doc = nlp(texto)
displacy.serve(doc, style="ent")

In [ ]:
nlp_en = spacy.load("en")
text = '''A trade war between the world’s two largest economies officially began on Friday morning as the Trump administration followed through with its threat to impose tariffs on $34 billion worth of Chinese products, a significant escalation of a fight that could hurt companies and consumers in both the United States and China.'''
doc = nlp_en(text)
displacy.serve(doc, style='ent')

## Similitud semántica entre palabras, frases y documentos

spaCy permite [calcular la similitud semántica](https://spacy.io/usage/vectors-similarity) entre cualquier par de objetos de tipo `Doc`, `Span` o `Token`. 

Ojo, La similitud semántica es un concepto algo subjetivo, pero en este caso se puede entender como la probabilidad de que dos palabras aparezcan en los mismos contextos.

In [8]:
# analizamos algunas colocaciones en inglés
nlp_en = spacy.load("en")
token1, _, token2 = nlp_en("cats and dogs")
token3, _, token4 = nlp_en("research and development")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

cats vs dogs 0.59674376
research vs development 0.5260883
cats vs development 0.12905318


/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instea

In [9]:
# ¿qué tal funciona en español?
nlp = spacy.load("es")
token1, _, token2 = nlp("perros y gatos")
token3, _, token4 = nlp("investigación y desarrollo")

# medimos la similitud semántica entre algunos pares
print(token1, "vs", token2, token1.similarity(token2))
print(token3, "vs", token4, token3.similarity(token4))
print(token1, "vs", token4, token1.similarity(token4))

perros vs gatos 0.8048168
investigación vs desarrollo 0.20338243
perros vs desarrollo -0.06565488


/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  "__main__", mod_spec)
/usr/lib/python3.7/runpy.py:193: ModelsWarning: [W007] The model you're using has no word vectors loaded, so the result of the Token.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instea